<a href="https://colab.research.google.com/github/SuriyaPriya17/image_processing/blob/main/sift_orb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#SIFT
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def build_gaussian_pyramid(image, num_octaves=4, scales_per_octave=5, sigma=1.6):
    k = 2**(1/scales_per_octave)
    pyramid = []
    for o in range(num_octaves):
        octave = [image]
        for s in range(1, scales_per_octave):
            sigma_prev = sigma * (k**(s-1))
            sigma_curr = sigma * (k**s)
            sigma_diff = np.sqrt(sigma_curr**2 - sigma_prev**2)
            blurred = cv2.GaussianBlur(octave[-1], (5, 5), sigmaX=sigma_diff, sigmaY=sigma_diff)
            octave.append(blurred)
        pyramid.append(octave)
        image = cv2.resize(octave[-1], (octave[-1].shape[1] // 2, octave[-1].shape[0] // 2))
    return pyramid

def build_dog_pyramid(gaussian_pyramid):
    dog_pyramid = []
    for octave in gaussian_pyramid:
        dog = []
        for i in range(1, len(octave)):
            dog.append(octave[i] - octave[i - 1])
        dog_pyramid.append(dog)
    return dog_pyramid

def detect_keypoints(dog_pyramid, threshold=0.03):
    keypoints = []
    for o, octave in enumerate(dog_pyramid):
        for i in range(1, len(octave) - 1):
            prev_img, curr_img, next_img = octave[i-1], octave[i], octave[i+1]
            for x in range(1, curr_img.shape[0] - 1):
                for y in range(1, curr_img.shape[1] - 1):
                    val = curr_img[x, y]
                    patch = np.concatenate([
                        prev_img[x-1:x+2, y-1:y+2].flatten(),
                        curr_img[x-1:x+2, y-1:y+2].flatten(),
                        next_img[x-1:x+2, y-1:y+2].flatten()
                    ])
                    if (val == patch.max() or val == patch.min()) and abs(val) > threshold:
                        keypoints.append((x * (2**o), y * (2**o), o))
    return keypoints

def compute_orientation(image, keypoints):
    oriented_keypoints = []
    gx = cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize=3)
    magnitude = np.sqrt(gx**2 + gy**2)
    angle = np.rad2deg(np.arctan2(gy, gx)) % 360
    for x, y, _ in keypoints:
        x, y = int(x), int(y)
        if 8 < x < image.shape[0]-8 and 8 < y < image.shape[1]-8:
            region_mag = magnitude[x-8:x+8, y-8:y+8]
            region_ang = angle[x-8:x+8, y-8:y+8]
            hist, _ = np.histogram(region_ang, bins=36, range=(0, 360), weights=region_mag)
            max_val = hist.max()
            peaks = np.where(hist > 0.8 * max_val)[0]
            for peak in peaks:
                dominant_angle = peak * 10
                oriented_keypoints.append((x, y, dominant_angle))
    return oriented_keypoints

def compute_descriptors(image, keypoints):
    gx = cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize=3)
    magnitude = np.sqrt(gx**2 + gy**2)
    angle = (np.rad2deg(np.arctan2(gy, gx)) + 360) % 360
    descriptors = []
    for x, y, orientation in keypoints:
        x, y = int(x), int(y)
        if x < 8 or y < 8 or x+8 >= image.shape[0] or y+8 >= image.shape[1]:
            continue
        patch_mag = magnitude[x-8:x+8, y-8:y+8]
        patch_ang = (angle[x-8:x+8, y-8:y+8] - orientation) % 360
        desc = []
        for i in range(0, 16, 4):
            for j in range(0, 16, 4):
                cell_mag = patch_mag[i:i+4, j:j+4]
                cell_ang = patch_ang[i:i+4, j:j+4]
                hist, _ = np.histogram(cell_ang, bins=8, range=(0, 360), weights=cell_mag)
                desc.extend(hist)
        desc = np.array(desc)
        descriptors.append(desc)
    return np.array(descriptors)

def sift_features(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    gaussian_pyramid = build_gaussian_pyramid(img)
    dog_pyramid = build_dog_pyramid(gaussian_pyramid)
    keypoints = detect_keypoints(dog_pyramid)
    oriented_keypoints = compute_orientation(img, keypoints)
    descriptors = compute_descriptors(img, oriented_keypoints)
    print("Total keypoints detected:", len(oriented_keypoints))
    return oriented_keypoints, descriptors

image_path = "sample.jpeg"
kps, desc = sift_features(image_path)
img = cv2.imread(image_path)
for (x, y, ang) in kps:
    cv2.circle(img, (int(y), int(x)), 2, (0, 255, 0), 1)
cv2_imshow(img)

In [ ]:
import numpy as np
import cv2
import math
from google.colab.patches import cv2_imshow

def detect_keypoints(image, threshold=30):
    h, w = image.shape
    keypoints = []
    offsets = np.array([
        (0, -3), (1, -3), (2, -2), (3, -1),
        (3, 0), (3, 1), (2, 2), (1, 3),
        (0, 3), (-1, 3), (-2, 2), (-3, 1),
        (-3, 0), (-3, -1), (-2, -2), (-1, -3)
    ])
    for y in range(3, h-3):
        for x in range(3, w-3):
            center = int(image[y, x]) # Cast to int to prevent overflow
            circle = np.array([int(image[y+dy, x+dx]) for dx, dy in offsets]) # Cast to int to prevent overflow
            brighter = np.sum(circle > center + threshold)
            darker = np.sum(circle < center - threshold)
            if brighter >= 12 or darker >= 12:
                keypoints.append((x, y))
    return keypoints

def compute_orientation(image, keypoints, patch_size=31):
    orientations = []
    half = patch_size // 2
    for (x, y) in keypoints:
        if x < half or y < half or x >= image.shape[1]-half or y >= image.shape[0]-half:
            orientations.append(0)
            continue

        patch = image[y-half:y+half+1, x-half:x+half+1]
        total_pixel_sum = np.sum(patch)
        if total_pixel_sum == 0:
            orientations.append(0)
            continue

        # Corrected moment calculations for orientation
        m10 = np.sum(np.arange(-half, half+1) * np.sum(patch, axis=0))
        m01 = np.sum(np.arange(-half, half+1)[:, np.newaxis] * np.sum(patch, axis=1)[:, np.newaxis])

        angle = math.atan2(m01, m10)
        orientations.append(angle)
    return orientations

def generate_brief_descriptors(image, keypoints, orientations, patch_size=31, n_bits=256):
    np.random.seed(42)
    half = patch_size // 2
    pattern = np.random.randint(-half, half, (n_bits, 4))
    descriptors = []
    h, w = image.shape
    for (x, y), theta in zip(keypoints, orientations):
        if x < half or y < half or x >= w - half or y >= h - half:
            descriptors.append(np.zeros(n_bits, dtype=np.uint8))
            continue
        cos_t, sin_t = np.cos(theta), np.sin(theta)
        desc = np.zeros(n_bits, dtype=np.uint8)
        for i, (x1, y1, x2, y2) in enumerate(pattern):
            xr1 = int(x1 * cos_t - y1 * sin_t)
            yr1 = int(x1 * sin_t + y1 * cos_t)
            xr2 = int(x2 * cos_t - y2 * sin_t)
            yr2 = int(x2 * sin_t + y2 * cos_t)

            # Boundary checks
            final_y1 = y + yr1
            final_x1 = x + xr1
            final_y2 = y + yr2
            final_x2 = x + xr2

            if final_y1 < 0 or final_y1 >= h or final_x1 < 0 or final_x1 >= w or \
               final_y2 < 0 or final_y2 >= h or final_x2 < 0 or final_x2 >= w:
                desc[i] = 0
            else:
                p1 = image[final_y1, final_x1]
                p2 = image[final_y2, final_x2]
                desc[i] = 1 if p1 < p2 else 0
        descriptors.append(desc)
    return np.array(descriptors, dtype=np.uint8)

def orb_feature_extraction(image):
    image = cv2.GaussianBlur(image, (5, 5), 1.2)
    keypoints = detect_keypoints(image)
    orientations = compute_orientation(image, keypoints)
    descriptors = generate_brief_descriptors(image, keypoints, orientations)
    return keypoints, descriptors

image = cv2.imread('sample.jpeg', cv2.IMREAD_GRAYSCALE)

keypoints, descriptors = orb_feature_extraction(image)

output = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
for (x, y) in keypoints:
    cv2.circle(output, (x, y), 2, (0, 255, 0), 1)

cv2_imshow(output)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
print(f"Extracted {len(keypoints)} keypoints")
